# 基础理论
* JaveEE: https://javaee.github.io/tutorial/toc.html
* <b>JMS concepts</b>: https://javaee.github.io/tutorial/jms-concepts.html#BNCDQ 
    * <b>What Is Messaging</b>: Messaging enables distributed communication that is loosely coupled. A component sends a message to a destination, and the recipient can retrieve the message from the destination. The sender and the receiver do not have to be available at the same time in order to communicate. In fact, the sender does not need to know anything about the receiver; nor does the receiver need to know anything about the sender. The sender and the receiver need to know only which message format and which destination to use.
    * <b>What Is the JMS API</b>: The Java Message Service is a Java API that allows applications to create, send, receive, and read messages. The JMS API defines a common set of interfaces and associated semantics that allow programs written in the Java programming language to communicate with other messaging implementations. 
        * Asynchronous: A receiving client does not have to receive messages at the same time the sending client sends them. The sending client can send them and go on to other tasks; the receiving client can receive them much later.
        * Reliable: A messaging provider that implements the JMS API can ensure that a message is delivered once and only once. Lower levels of reliability are available for applications that can afford to miss messages or to receive duplicate messages.
* <b>JMS API Architecture:</b>
    * <b>A JMS provider</b> is a messaging system that implements the JMS interfaces and provides administrative and control features. An implementation of the Java EE platform that supports the full profile includes a JMS provider. such as `ActiveMQ or RabbitMQ`
    * <b>JMS clients</b> are the programs or components, written in the Java programming language, that produce and consume messages. Any Java EE application component can act as a JMS client.
    * <b>Messages</b> are the objects that communicate information between JMS clients.
    * <b>Administered objects</b> are JMS objects configured for the use of clients. The two kinds of JMS administered objects are destinations and connection factories, described in JMS Administered Objects. An administrator can create objects that are available to all applications that use a particular installation of GlassFish Server; alternatively, a developer can use annotations to create objects that are specific to a particular application.
    * <b><i>Summary: Administrative tools or annotations allow you to bind destinations and connection factories into a JNDI namespace. A JMS client can then use resource injection to access the administered objects in the namespace and then establish a logical connection to the same objects through the JMS provider.</i></b>
    <img src="../../images/javaee/javaeett_dt_027.png" width="400px">
* <b>Messaging Styles</b>
    * <i>`Point-to-Point Messaging Style`</i>: A point-to-point (PTP) product or application is built on the concept of message queues, senders, and receivers. Each message is addressed to a specific queue, and receiving clients extract messages from the queues established to hold their messages. Queues retain all messages sent to them until the messages are consumed or expire.
        * Each message has only one consumer.
        * The receiver can fetch the message whether or not it was running when the client sent the message.
        * <img src="../../images/javaee/javaeett_dt_028.png" width="400px">
    * <i>`Publish/Subscribe Messaging Style`</i>: In a publish/subscribe (pub/sub) product or application, clients address messages to a topic, which functions somewhat like a bulletin board. Publishers and subscribers can dynamically publish or subscribe to the topic. The system takes care of distributing the messages arriving from a topic’s multiple publishers to its multiple subscribers. Topics retain messages only as long as it takes to distribute them to subscribers.
        * With pub/sub messaging, it is important to distinguish between the consumer that subscribes to a topic (the subscriber) and the subscription that is created. The consumer is a JMS object within an application, while the subscription is an entity within the JMS provider. Normally, a topic can have many consumers, but a subscription has only one subscriber.
        * Each message can have multiple consumers.
        * A client that subscribes to a topic can consume only messages sent after the client has created a subscription, and the consumer must continue to be active in order for it to consume messages.
        * <img src="../../images/javaee/javaeett_dt_029.png" width="400px">
* <b>Message Consumption</b>
    * Synchronously: A consumer explicitly fetches the message from the destination by calling the receive method. The receive method can block until a message arrives or can time out if a message does not arrive within a specified time limit.
    * Asynchronously: An application client or a Java SE client can register a message listener with a consumer. A message listener is similar to an event listener. Whenever a message arrives at the destination, the JMS provider delivers the message by calling the listener’s onMessage method, which acts on the contents of the message. In a Java EE application, a message-driven bean serves as a message listener (it too has an onMessage method), but a client does not need to register it with a consumer.

****

# 应用实战-编程模型
* 示例参考: https://javaee.github.io/tutorial/jms-examples.html#BNCGV
* 设计总图: https://javaee.github.io/tutorial/jms-concepts003.html
<img src="../../images/javaee/javaeett_dt_030.png" width="400px">
* <b>JMS Administered Objects</b>:
    * <i>Two parts of a JMS application, destinations and connection factories, are commonly maintained administratively rather than programmatically. The technology underlying these objects is likely to be very different from one implementation of the JMS API to another. Therefore, the management of these objects belongs with other administrative tasks that vary from provider to provider.</i>
    * <b>JMS Connection Factories</b>: A connection factory is the object a client uses to create a connection to a provider. A connection factory encapsulates a set of connection configuration parameters that has been defined by an administrator. Each connection factory is an instance of the `ConnectionFactory`, `QueueConnectionFactory`, or `TopicConnectionFactory` interface. 
        * `@Resource(lookup = "java:comp/DefaultJMSConnectionFactory") private static ConnectionFactory connectionFactory;`
    * <b>JMS Destinations</b>: A destination is the object a client uses to specify the target of messages it produces and the source of messages it consumes. In the PTP messaging style, destinations are called queues. In the pub/sub messaging style, destinations are called topics. A JMS application can use multiple queues or topics (or both). 
        * `@Resource(lookup = "jms/MyQueue") private static Queue queue; @Resource(lookup = "jms/MyTopic") private static Topic topic;`
* <b>Connections</b>: A connection encapsulates a virtual connection with a JMS provider. For example, a connection could represent an open TCP/IP socket between a client and a provider service daemon. You use a connection to create one or more sessions.
* <b>Sessions</b>: A session is a single-threaded context for producing and consuming messages. You normally create a session by creating a JMSContext object. You use sessions to create message producers, message consumers, messages, queue browsers, and temporary destinations.
* <b>JMSContext Objects</b>: A JMSContext object combines a connection and a session in a single object. That is, it provides both an active connection to a JMS provider and a single-threaded context for sending and receiving messages.
    * `JMSContext context = connectionFactory.createContext();`
    * `JMSContext context = connectionFactory.createContext(JMSContext.SESSION_TRANSACTED);`
* <b>JMS Message Producers</b>: A message producer is an object that is created by a JMSContext or a session and used for sending messages to a destination. A message producer created by a JMSContext implements the JMSProducer interface.
    * `JMSProducer producer = context.createProducer();`
    * `context.createProducer().send(dest, message);`
* <b>JMS Message Consumers</b>: A message consumer is an object that is created by a JMSContext or a session and used for receiving messages sent to a destination. A message producer created by a JMSContext implements the JMSConsumer interface. 
    * `JMSConsumer consumer = context.createConsumer(dest);`
    * A message consumer allows a JMS client to register interest in a destination with a JMS provider. The JMS provider manages the delivery of messages from a destination to the registered consumers of the destination.
    * When you use a JMSContext to create a message consumer, message delivery begins as soon as you have created the consumer. You can disable this behavior by calling setAutoStart(false) when you create the JMSContext and then calling the start method explicitly to start message delivery. If you want to stop message delivery temporarily without closing the connection, you can call the stop method; to restart message delivery, call start.
    * <i>Synchronously consumption</i>:
        * `Message m = consumer.receive(); Message m = consumer.receive(0); Message m = consumer.receive(1000);`
    * <i>Asynchronously consumption</i>:
        * JMS Message Listeners: A message listener is an object that acts as an asynchronous event handler for messages. This object implements the MessageListener interface, which contains one method, onMessage. In the onMessage method, you define the actions to be taken when a message arrives.
        * `Listener myListener = new Listener(); consumer.setMessageListener(myListener);`
    * JMS Message Selectors: If your messaging application needs to filter the messages it receives, you can use a JMS message selector, which allows a message consumer for a destination to specify the messages that interest it. Message selectors assign the work of filtering messages to the JMS provider rather than to the application. 
    * <b>Consuming Messages from Topics</b>
        * An application consumes messages from a topic by creating a subscription on that topic and creating a consumer on that subscription. Subscriptions may be durable or nondurable, and they may be shared or unshared.
        * Subscription may be thought of as an entity within the JMS provider itself, whereas a consumer is a JMS object within the application.
        * `Unshared subscriptions` are restricted to a single consumer. All the messages in the subscription are delivered to that consumer.
        * `Shared subscriptions` allow multiple consumers. Each message in the subscription is delivered to only one consumer. 
        * `A nondurable subscription` exists only as long as there is an active consumer on the subscription. This means that any messages sent to the topic will be added to the subscription only while a consumer exists and is not closed.
            * `An unshared nondurable subscription` does not have a name and may have only a single consumer object associated with it. It is created automatically when the consumer object is created. It is not persisted and is deleted automatically when the consumer object is closed.
            * `A shared nondurable subscription` is identified by name and an optional client identifier, and may have several consumer objects consuming messages from it. It is created automatically when the first consumer object is created. It is not persisted and is deleted automatically when the last consumer object is closed.
            * `JMSContext.createConsumer`
        * `A durable subscription` is persisted and continues to accumulate messages until explicitly deleted, even if there are no consumer objects consuming messages from it.
            * `An unshared durable subscription` is identified by name and client identifier (which must be set) and may have only a single consumer object associated with it.
            * `A shared durable subscription` is identified by name and an optional client identifier, and may have several consumer objects consuming messages from it.
            * `JMSContext.createDurableConsumer`
            * `JMSConsumer consumer = context.createSharedDurableConsumer(topic, "MakeItLast");`
            * `consumer = context.createSharedConsumer(topicName, "SubName");`
* <b>JMS Messages</b>
    * Message Headers: A JMS message header contains a number of predefined fields that contain values used by both clients and providers to identify and route messages.
    * Message Properties: You can create and set properties for messages if you need values in addition to those provided by the header fields. You can use properties to provide compatibility with other messaging systems, or you can use them to create message selectors.
    * Message Bodies: The JMS API defines six different types of messages. Each message type corresponds to a different message body. 
* <b>Advanced Message Queuing Protocol, AMQP</b>
    * The JMS client API is standardized, so JMS applications are portable between vendors’ implementations. However, the underlying messaging implementation is unspecified, so there is no interoperability between JMS implementations. Java applications that want to share messaging must all use the same JMS implementation unless bridging technology exists. Furthermore, non-Java applications cannot access JMS without a vendor-specific JMS client library to enable interoperability.
    * AMQP is a messaging protocol, rather than an API like JMS. Any client that implements the protocol can access a broker that supports AMQP. Protocol-level interoperability allows AMQP clients written in any programming language and running on any operating system to participate in the messaging system with no need to bridge incompatible vendor implementations.

****

# JMS Providers
* ActiveMQ (实现思路参考JMS): http://activemq.apache.org/
* ActiveMQ Architecture: <img src="../../images/javaee/activeMQ.png" width="450px">
* RabbitMQ (实现思路参考JMS): https://www.rabbitmq.com/
    * 代码实战示例: https://www.rabbitmq.com/getstarted.html
    * 服务端配置示例: https://www.rabbitmq.com/admin-guide.html
    * 客户端接口示例: https://www.rabbitmq.com/clients.html